In [ ]:
# Importing the necessary libraries

import tensorflow as tf
import albumentations as albu
import numpy as np
import gc
import pickle
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, precision_score, recall_score, accuracy_score, f1_score
import pandas as pd

In [ ]:
from zipfile import ZipFile
import os

zip_file_name = '/content/DUCK-Net-main.zip'
model_folder_path = '/content/DUCK-Net-main'

if not os.path.exists(model_folder_path):
    os.makedirs(model_folder_path)

with ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(model_folder_path)

In [ ]:
import sys
sys.path.append('/content/DUCK-Net-main/CustomLayers')
import ConvBlock2D
sys.path.append('/content/DUCK-Net-main/ModelArchitecture')
import DiceLoss
from DiceLoss import dice_metric_loss
import DUCK_Net
sys.path.append('/content/DUCK-Net-main/ImageLoader')
import ImageLoader2D

In [ ]:
# Checking the number of GPUs available

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Setting the model parameters

img_size = 352
dataset_type = 'kvasir' # Options: kvasir/cvc-clinicdb/cvc-colondb/etis-laribpolypdb
learning_rate = 1e-4
seed_value = 58800
filters = 34 # Number of filters, the paper presents the results with 17 and 34
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

model_type = "DuckNet"

min_loss_for_saving = 0.2

/content/ProgressFull/kvasir_progress_csv_DuckNet_filters_34_2024-03-23 15:25:18.078383.csv
/content/ProgressFull/kvasir_progress_DuckNet_filters_34_2024-03-23 15:25:18.078383.txt
/content/ProgressFull/kvasir_progress_plot_DuckNet_filters_34_2024-03-23 15:25:18.078383.png
/content/ModelSaveTensorFlow/kvasir/DuckNet_filters_34_2024-03-23 15:25:18.078383


In [ ]:
from zipfile import ZipFile
import os

zip_file_name = '/content/kvasir-seg.zip'
dataset_path = '/content/'

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)

with ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

dataset_path +='/Kvasir-SEG/'

In [ ]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive/')
saved_model_path = Path('/content/drive/My Drive/Computer Vision/DuckNet34 Kvasir Tf Model')


Mounted at /content/drive/


In [ ]:
# Loading the data

X, Y = ImageLoader2D.load_data(img_size, img_size, -1, 'kvasir', dataset_path)

Resizing training images and masks: 1000


1000it [16:23,  1.02it/s]


In [ ]:
# Load the model weight

print("Loading the model")

model = tf.keras.models.load_model(saved_model_path, custom_objects={'dice_metric_loss':dice_metric_loss})

Loading the model


In [ ]:
prediction_test = model.predict(X, batch_size=4)
print("Predictions done")


dice_test = f1_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
print("Dice finished")


miou_test = jaccard_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                          np.ndarray.flatten(prediction_test > 0.5))
print("Miou finished")

# Save the DataFrame to an Excel file
dice_scores = []
miou_scores = []

for i in range(len(Y)):
    dice_score = f1_score(np.ndarray.flatten(np.array(Y[i], dtype=bool)),
                          np.ndarray.flatten(prediction_test[i] > 0.5))
    miou_score = jaccard_score(np.ndarray.flatten(np.array(Y[i], dtype=bool)),
                                np.ndarray.flatten(prediction_test[i] > 0.5))
    dice_scores.append(dice_score)
    miou_scores.append(miou_score)

data = {'Image': range(len(Y)), 'Dice': dice_scores, 'MIOU': miou_scores}
image_metrics_df = pd.DataFrame(data)

image_metrics_df.to_excel("image-metrics-DUCKNet-train-kvasir-test-kvasir.xlsx", index=False)

print("Excel file with image-wise metrics generated successfully.")


precision_test = precision_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                                 np.ndarray.flatten(prediction_test > 0.5))
print("Precision finished")


recall_test = recall_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                           np.ndarray.flatten(prediction_test > 0.5))
print("Recall finished")


accuracy_test = accuracy_score(np.ndarray.flatten(np.array(Y, dtype=bool)),
                               np.ndarray.flatten(prediction_test > 0.5))
print("Accuracy finished")

final_file = "results-DUCKNet-train-kvasir-test-kvasir.txt"
print(final_file)

with open(final_file, 'a') as f:
    f.write(dataset_type + '\n\n')
    f.write(' dice_test: ' + str(dice_test) + '\n\n')
    f.write(' miou_test: ' + str(miou_test) + '\n\n')
    f.write(' precision_test: ' + str(precision_test) + '\n\n')
    f.write(' recall_test: ' + str(recall_test) + '\n\n')
    f.write(' accuracy_test: ' + str(accuracy_test) + '\n\n\n\n')

print('File done')

250/250 [==============================] - 184s 665ms/step
[[[[6.57130784e-07]
   [2.68567277e-07]
   [4.97846827e-07]
   ...
   [2.35069564e-08]
   [5.31092041e-08]
   [1.23084658e-06]]

  [[1.25176342e-07]
   [1.48744874e-07]
   [3.71689964e-07]
   ...
   [1.27065392e-09]
   [1.53015337e-10]
   [1.47161714e-08]]

  [[3.95088335e-08]
   [8.82224427e-08]
   [5.00925353e-07]
   ...
   [2.70486900e-09]
   [1.40309556e-10]
   [2.41927034e-09]]

  ...

  [[5.99645134e-08]
   [3.13140305e-08]
   [3.62581112e-08]
   ...
   [2.45387910e-09]
   [2.93476354e-09]
   [2.22891590e-08]]

  [[6.25640268e-07]
   [1.82238026e-07]
   [2.04059731e-07]
   ...
   [5.93769645e-10]
   [3.16508952e-08]
   [1.11674744e-07]]

  [[2.01782336e-06]
   [8.25986035e-07]
   [1.93782427e-07]
   ...
   [2.67705715e-08]
   [1.24468920e-06]
   [6.32947877e-06]]]


 [[[2.52118809e-07]
   [1.52698931e-07]
   [4.24908507e-07]
   ...
   [1.69170935e-08]
   [2.72350196e-08]
   [7.98507529e-07]]

  [[1.01728730e-07]
   [8.843